In [1]:
%load_ext autoreload
%autoreload 2
import os


## Activate fmriprep

Firstly, we need to activate fmriprep. This will add the relevant files to our path. This only needs doing once (you dont have to do it again next time you login).



In [2]:
from CINN_fmriprep.fmriprep import activate_fmriprep
activate_fmriprep()

fmriprep already activated ~/.bashrc


## Define an fmriprep handler.

Here I define an fmriprephandler. The main arguments are the path to the BIDS directory on which to do the fmriprepping, the output directory and the work directory for intermediate files. slurmout_path will define where the files that update on progress and errors will be stored.  

In this case, I also define a subject on whom to do the fmriprepping. If this argument is not included it defaults to doing all subjects (not reccommended).

In [3]:
from CINN_fmriprep.fmriprep import FmriPrepHandler as FPH
k= FPH(bids_path='/storage/basic/nh_leverhulme/Richardson_2018',out_path='/storage/basic/nh_leverhulme/fmriprep',work_path='/storage/basic/nh_leverhulme/work',
       slurmout_path='/home/users/yg916972/Scripts/',subject='sub-pixar007')


I now make the SLURM job for the subject using the make_slurm method.

In [4]:
k.make_slurm()

Now we have an fmriprep template script created. 

In [5]:
print(k.slurm)

#!/bin/sh

#SBATCH --cpus-per-task       16
#SBATCH --error               /home/users/yg916972/Scripts/fmriprep_sub-pixar007.err
#SBATCH --mem-per-cpu         4G
#SBATCH --nodes               1-1
#SBATCH --output              /home/users/yg916972/Scripts/fmriprep_sub-pixar007.out

source ~/.bashrc
cd /storage
export SINGULARITYENV_TEMPLATEFLOW_HOME=/storage/research/cinn_comp/cnbpy_Resources/templateflow



Upon being submitted, the following command will be submitted to the job queue using sbatch

In [59]:
k.cmd

'fmriprep /storage/basic/nh_leverhulme/Richardson_2018 /storage/basic/nh_leverhulme/fmriprep/sub-pixar007 participant --participant-label sub-pixar007 --fs-license-file /storage/research/cinn_comp/cnbpy_Resources/license.txt -w /storage/basic/nh_leverhulme/work/sub-pixar007 --output-spaces MNI152NLin2009cAsym '

If I wanted to add some additional arguments or output spaces to fmriprep, I could do so as follows

In [69]:
k.make_slurm(additionals=['--force-bbr'],output_spaces=['fsnative','fsaverage'])

In [70]:
k.cmd

'fmriprep /storage/basic/nh_leverhulme/Richardson_2018 /storage/basic/nh_leverhulme/fmriprep/sub-pixar007 participant --participant-label sub-pixar007 --fs-license-file /storage/research/cinn_comp/cnbpy_Resources/license.txt -w /storage/basic/nh_leverhulme/work/sub-pixar007 --output-spaces fsnative fsaverage --force-bbr'

The SLURM/ SBATCH pre-commands for provisioning resources are sensible defaults, but If I wanted to add in more, or different slurm pre-commands, I could do this by adding these to the 'slurm' subdictionary of the yml file that comes with the package.

In [54]:
k.y['slurm']

{'cpus_per_task': 16, 'mem_per_cpu': '4G', 'N': '1-1'}

In other words, I would modify this file:

In [55]:
k.yaml

'/home/users/yg916972/Software/CINN_fmriprep/config/config.yml'

The job can be submitted as follows. A message is printed to point to the location of the .out and .err files, which will contain outputs and errors respectively.

k.submit_slurm()

## Submitting multiple jobs in parallel. 

What we may often want to do is submit multiple subjects as multiple jobs to run in parallel. This process is mostly the same as before, but this time we can use the MultipleFmriPrepHandler. Here, we specify the additional argument of 'subjects' which is passed as a list of strings.

In [75]:
from CINN_fmriprep.fmriprep import MultipleFmriPrepHandler as MFPH

In [78]:
k= MFPH('/storage/basic/nh_leverhulme/Richardson_2018','/storage/basic/nh_leverhulme/fmriprep','/storage/basic/nh_leverhulme/work',
        slurmout_path='/home/users/yg916972/Scripts/',subjects=['sub-pixar007','sub-pixar008'])

In [79]:
k.make_fmriprep_handlers()
k.make_slurms(additionals=['--fs-no-reconall'])

Now we have a list of FmriPrepHandlers (one for each subject) which correspond to different jobscripts and commands. 

In [81]:
print(k.handlers[0].slurm)

#!/bin/sh

#SBATCH --cpus-per-task       16
#SBATCH --error               /home/users/yg916972/Scripts/fmriprep_sub-pixar007.err
#SBATCH --mem-per-cpu         4G
#SBATCH --nodes               1-1
#SBATCH --output              /home/users/yg916972/Scripts/fmriprep_sub-pixar007.out

source ~/.bashrc
cd /storage
export SINGULARITYENV_TEMPLATEFLOW_HOME=/storage/research/cinn_comp/cnbpy_Resources/templateflow



In [80]:
k.handlers[0].cmd

'fmriprep /storage/basic/nh_leverhulme/Richardson_2018 /storage/basic/nh_leverhulme/fmriprep/sub-pixar007 participant --participant-label sub-pixar007 --fs-license-file /storage/research/cinn_comp/cnbpy_Resources/license.txt -w /storage/basic/nh_leverhulme/work/sub-pixar007 --output-spaces MNI152NLin2009cAsym --fs-no-reconall'

In [82]:
print(k.handlers[1].slurm)

#!/bin/sh

#SBATCH --cpus-per-task       16
#SBATCH --error               /home/users/yg916972/Scripts/fmriprep_sub-pixar008.err
#SBATCH --mem-per-cpu         4G
#SBATCH --nodes               1-1
#SBATCH --output              /home/users/yg916972/Scripts/fmriprep_sub-pixar008.out

source ~/.bashrc
cd /storage
export SINGULARITYENV_TEMPLATEFLOW_HOME=/storage/research/cinn_comp/cnbpy_Resources/templateflow



In [83]:
print(k.handlers[1].cmd)

fmriprep /storage/basic/nh_leverhulme/Richardson_2018 /storage/basic/nh_leverhulme/fmriprep/sub-pixar008 participant --participant-label sub-pixar008 --fs-license-file /storage/research/cinn_comp/cnbpy_Resources/license.txt -w /storage/basic/nh_leverhulme/work/sub-pixar008 --output-spaces MNI152NLin2009cAsym --fs-no-reconall


In [43]:
k.submit_slurms()

Submitted batch job 70733

fMRIprep Job with fmriprep_sub-pixar007 submitted! 

               Outputs will be found at /storage/basic/nh_leverhulme/fmriprep/sub-pixar007 
 
              Progress updated in /home/users/yg916972/Scripts/fmriprep_sub-pixar007.out

              Errors will be reported at /home/users/yg916972/Scripts/fmriprep_sub-pixar007.err

Submitted batch job 70734

fMRIprep Job with fmriprep_sub-pixar008 submitted! 

               Outputs will be found at /storage/basic/nh_leverhulme/fmriprep/sub-pixar008 
 
              Progress updated in /home/users/yg916972/Scripts/fmriprep_sub-pixar008.out

              Errors will be reported at /home/users/yg916972/Scripts/fmriprep_sub-pixar008.err

